In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/undersampledrenn/Repeated_Edited_Nearest_Neighbors_Under-sampled_Dataset.csv


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/undersampledrenn/Repeated_Edited_Nearest_Neighbors_Under-sampled_Dataset.csv')
df = df[df['Label'] != 'BENIGN']

In [ ]:
df.shape

# From The paper
Build a LSTM Neural Network with the following description: three LSTM layers with 128 unit and sigmoid as 
activation function, three dropout layers, and a dense 
layer with tanh function. Use categorical_crossentropy as loss function and Root 
Mean Square Propagation (rmsprop) as an optimizer

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam


from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense


# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = to_categorical(labels)


# Define the LSTM model
# define the model
model = Sequential()
model.add(LSTM(units=128, activation='sigmoid', return_sequences=True, input_shape=(features.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=128, activation='sigmoid', return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(labels.shape[1], activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# print the model summary
model.summary()

# Define the number of folds
n_splits = 5

# Initialize lists to store results
acc_scores = []
loss_scores = []

# Split the data into training and testing sets
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in skf.split(features,labels.argmax(1)):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Reshape the data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))



    # Train the model
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Evaluate the model
    score = model.evaluate(X_test, y_test, verbose=0)
    loss_scores.append(score[0])
    acc_scores.append(score[1])

# Print the average accuracy and loss
print(f'Average accuracy across {n_splits} folds: {np.mean(acc_scores):.2f}')
print(f'Average loss across {n_splits} folds: {np.mean(loss_scores):.2f}')


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 42, 128)           66560     
                                                                 
 dropout (Dropout)           (None, 42, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 42, 128)           131584    
                                                                 
 dropout_1 (Dropout)         (None, 42, 128)           0         
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 11)                1

# Better

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam



# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = to_categorical(labels)

# Define the number of folds
n_splits = 5

# Initialize lists to store results
acc_scores = []
loss_scores = []


# Define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(features.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
#model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(labels.shape[1], activation="softmax"))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

# Split the data into training and testing sets
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in skf.split(features,labels.argmax(1)):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Reshape the data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    

    # Train the model
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)

    # Evaluate the model
    score = model.evaluate(X_test, y_test, verbose=0)
    loss_scores.append(score[0])
    acc_scores.append(score[1])

# Print the average accuracy and loss
print(f'Average accuracy across {n_splits} folds: {np.mean(acc_scores):.2f}')
print(f'Average loss across {n_splits} folds: {np.mean(loss_scores):.2f}')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam



# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = to_categorical(labels)

# Define the number of folds
n_splits = 5

# Initialize lists to store results
acc_scores = []
loss_scores = []

# Split the data into training and testing sets
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
for train_index, test_index in skf.split(features,labels.argmax(1)):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Reshape the data for LSTM
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    model = Sequential()
    model.add(LSTM(128,input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add((Dense(labels.shape[1], activation='softmax')))

    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-4), metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test), verbose=0)

    # Evaluate the model
    score = model.evaluate(X_test, y_test, verbose=0)
    loss_scores.append(score[0])
    acc_scores.append(score[1])

# Print the average accuracy and loss
print(f'Average accuracy across {n_splits} folds: {np.mean(acc_scores):.2f}')
print(f'Average loss across {n_splits} folds: {np.mean(loss_scores):.2f}')
